<a href="https://colab.research.google.com/github/timlovescoding/FASHION-MNIST/blob/master/GAN_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## Learning about GAN from book written by Francois Chollet


# 1) Building the generator:

import keras
from keras import layers
from keras import models

latent_dim = 32 #dimension of latent space

# Generate 32x32x3 image~
height   = 32
width    = 32
channels = 3

#Setting up Neural Network for Generator:

gen_input = keras.Input(shape=(latent_dim,))  #Input to generator

x = layers.Dense(16*16*128)(gen_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16,16,128))(x)

x = layers.Conv2D(256,5, padding = 'same')(x) # 5x5 Kernel, 256 filters, same padding
x = layers.LeakyReLU()(x)
 
x = layers.Conv2DTranspose(256,4,strides = 2, padding = 'same')(x)   #Upsampling (Reverse Convolution)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256,5,padding = 'same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256,5,padding = 'same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels,7,activation = 'tanh', padding='same')(x)

generator = models.Model(gen_input,x)
generator.summary()

Using TensorFlow backend.
W0825 12:27:56.299717 139799833900928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0825 12:27:56.334737 139799833900928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0825 12:27:56.343086 139799833900928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

In [0]:
# 2) Building the discriminator:


dis_input = layers.Input(shape=(height,width,channels))

x = layers.Conv2D(128,3)(dis_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4,strides = 2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4,strides = 2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4,strides = 2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x) # important to use dropout or else gradient may vanished easily

x = layers.Dense(1,activation = 'sigmoid')(x) # 0 or 1 Classification (Fake or Real)

discriminator = models.Model(dis_input,x)
discriminator.summary()

from keras import optimizers

dis_optimizer = optimizers.RMSprop(lr = 0.0008, clipvalue = 1.0, decay =1e-8)
discriminator.compile(optimizer = dis_optimizer ,  loss = "binary_crossentropy")

W0825 12:27:56.511124 139799833900928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0825 12:27:56.517534 139799833900928 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0825 12:27:56.552984 139799833900928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0825 12:27:56.557825 139799833900928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tenso

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 6, 6, 128)         0         
__________

In [0]:
# 3) The adversarial network: (Generator trying to fool the discriminator)


discriminator.trainable = False #Applying only to the GAN model (Do not want to train discriminator to always predict REAL)

gan_input = keras.Input(shape = (latent_dim,)) 

gan_output = discriminator(generator(gan_input)) #Generated data feed into discriminator

gan = models.Model(gan_input, gan_output)

gan_optimizer = optimizers.RMSprop( lr=0.0004, clipvalue = 1.0, decay = 1e-8)
gan.compile(optimizer =  gan_optimizer, loss ="binary_crossentropy") #Output is discriminator (binary loss function)



gan.summary() #Trainable Params: The generator.
              #Non-trainable params: The discriminator


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32)                0         
_________________________________________________________________
model_1 (Model)              (None, 32, 32, 3)         6264579   
_________________________________________________________________
model_2 (Model)              (None, 1)                 790913    
Total params: 7,055,492
Trainable params: 6,264,579
Non-trainable params: 790,913
_________________________________________________________________


In [0]:
# Training the GAN:

# 1. Draw random points in the latent space
# 2. Generate images with generator using these random points (Generated Image Set)
# 3. Mix the generated images with real ones (Mixed Image set)
# 4. Train discriminator using the Mixed Image set (Binary loss function: Real or Fake)
# 5. Draw new random points in the latent space
# 6. Train the adversial network using these random vectors with target to Real Images
#    -Update weights of generator till the generator is able to fool the discriminator
#    -Discriminator weight is frozen in GAN model

###################################################################

# 4) Getting the datasets for real images:


from keras.datasets import cifar10

(x_train , y_train) , (_,_) = cifar10.load_data()

print("Training size" + str(x_train.shape))
print("Label size" + str(y_train.shape))
print(y_train.flatten())


170500096/170498071 [==============================] - 2s 0us/step
Training size(50000, 32, 32, 3)
Label size(50000, 1)
[6 9 9 ... 9 1 1]


In [0]:
# Getting real images from CIFAR 10

x_train = x_train[ y_train.flatten() == 6 ] #Class 6 is all of the frogs

x_train= x_train.reshape((x_train.shape[0],) + (height,width,channels)) # 5000 samples (32x32x3)

x_train= x_train/255.0 # Normalize

In [0]:
# 5) Setting up training

import numpy as np
import os
from keras.preprocessing import image


from google.colab import drive
drive.mount('/content/gdrive')

my_dir = '/content/gdrive/My Drive/GAN_IMAGES_1'   #Set up your own directory in Google Drive (using colaboratory to run my code)

iterations = 15000
batch_size = 50

start = 0

for step in range(iterations):
  
  random_latent_vectors = np.random.normal(size =(batch_size,latent_dim))  # Samples of random points in the latent space
  
  generated_images =  generator.predict(random_latent_vectors)  #Decoding into fake images through GENERATOR NN
# Generated images are of batch size 20x32x32x3

  stop = start  + batch_size
  real_images = x_train[start:stop]
  combined_images = np.concatenate([generated_images, real_images],axis=0) # Shape = (40,32,32,3) 40 because 20+20 from both image set


#Fixing the label onto the images:

  labels = np.concatenate([np.ones((generated_images.shape[0],1)) , np.zeros((real_images.shape[0],1))]) # 1 for generated, 0 for real
  # Label shape = (40,1)

#*** Add random noise into labels (Trick for training)

  labels = labels + 0.065*np.random.random(labels.shape)
  
# 6) Train the Discriminator:

  dis_loss = discriminator.train_on_batch(combined_images,labels)


  random_latent_vectors = np.random.normal(size= (batch_size,latent_dim))
  
  misleading_targets = np.zeros((batch_size,1))
  
  adv_loss = gan.train_on_batch(random_latent_vectors,misleading_targets) #note: discriminator is frozen in GAN Model
  
  
  start = start + batch_size
  
  if start > (len(x_train) - batch_size):
    start = 0
    
  if step % 1000 == 0:
    
    gan.save_weights('gan.h5')
    
    print('Discriminator Loss:' , dis_loss)
    print('Adversarial Loss:', adv_loss)
    
    img =  image.array_to_img(generated_images[0] * 255.0, scale = False)
    img.save(os.path.join(my_dir,'generated_frog' + str(step)+ '.png'))
    
 
    img =  image.array_to_img(real_images[0] * 255.0, scale = False)
    img.save(os.path.join(my_dir,'real_frog' + str(1)+ '.png'))
    
    
    
    
   


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Discriminator Loss: 0.6805521
Adversarial Loss: 0.6801954


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Discriminator Loss: 0.69642174
Adversarial Loss: 0.7566668
Discriminator Loss: 0.69223726
Adversarial Loss: 0.78600234
Discriminator Loss: 0.69282943
Adversarial Loss: 0.77431357
Discriminator Loss: 0.6945237
Adversarial Loss: 0.7724354
Discriminator Loss: 0.7035577
Adversarial Loss: 0.7865812
Discriminator Loss: 0.68808055
Adversarial Loss: 0.764144
Discriminator Loss: 0.68804413
Adversarial Loss: 0.7524828
Discriminator Loss: 0.69160044
Adversarial Loss: 0.7663623
Discriminator Loss: 0.69707
Adversarial Loss: 0.7428724
Discriminator Loss: 0.69034934
Adversarial Loss: 0.76276034
Discriminator Loss: 0.68691176
Adversarial Loss: 0.78465956
Discriminator Loss: 0.6928648
Adversarial Loss: 0.7506774
Discriminator Loss: 0.7031819
Adversarial Loss: 0.84559166
Discriminator Loss: 0.7062076
Adversarial Loss: 0.8469773
